In [5]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Tab, Label, Checkbox, Button
from ipywidgets import FloatSlider, IntSlider, Dropdown, SelectMultiple
from IPython.display import display

import matplotlib.pyplot as plt
import matplotlib; matplotlib.rcParams.update({'font.size': 14})
import seaborn as sns; sns.set_style('whitegrid')
import numpy as np

import pandas as pd
from datetime import datetime

In [6]:
df = pd.read_parquet("cmm_erdos_bootcamp_2020_timeseries.pq", engine='pyarrow')
df.date_val = pd.to_datetime(df.date_val)

test_start_dates = {'1 month': datetime(2019, 12, 1),
                    '3 months': datetime(2019, 10, 1),
                    '6 months': datetime(2019, 7, 1),
                    '1 year': datetime(2019, 1, 1)}

def get_train_test(test_start_date):
    df_train = df.loc[df.date_val < test_start_date]
    df_test = df.loc[df.date_val >= test_start_date]
    
    train_a, train_b, train_c = df_train.volume_A.values, df_train.volume_B.values, df_train.volume_C.values
    test_a, test_b, test_c = df_test.volume_A.values, df_test.volume_B.values, df_test.volume_C.values

    train_sets = {'Volume A': train_a,
                 'Volume B': train_b,
                 'Volume C': train_c}

    test_sets = {'Volume A': test_a,
                'Volume B': test_b,
                'Volume C': test_c}
    
    return train_sets, test_sets, df_test.date_val

In [7]:
def mape(actual, prediction):
    return np.mean(np.abs((actual - prediction) / actual))

def normalized(vals):
    return (vals - vals.mean()) / (vals.max() - vals.min())

# Explore Data

In [8]:
from statsmodels.tsa.seasonal import STL

## Trend

In [9]:
colors = {'Volume A': 'C1',
          'Volume B': 'C2',
          'Volume C': 'C3'}

def plot_trends(volumes, normalize=True, reg_holidays=True, holiday_dates=True):
    if len(volumes) == 0:
        return
    
    # fetch data from dataframe
    vol_a = df.volume_A.values.copy()
    vol_b = df.volume_B.values.copy()
    vol_c = df.volume_C.values.copy()

    # preprocess if requested
    if reg_holidays:
        for i in df.index[df.is_holiday == 1]:
            if i >= 7:
                source_i = i - 7
            else:
                source_i = i + 7
            vol_a[i] = vol_a[source_i]
            vol_b[i] = vol_b[source_i]
            vol_c[i] = vol_c[source_i]

    # extract trends using STL
    trend_a = STL(vol_a, 7).fit().trend
    trend_b = STL(vol_b, 7).fit().trend
    trend_c = STL(vol_c, 7).fit().trend
    trends = {'Volume A': trend_a,
              'Volume B': trend_b,
              'Volume C': trend_c}
    
    
    plt.figure(figsize=(20, 5))
    for volume in volumes:
        trend = trends[volume]
        if normalize:
            trend = normalized(trend)
        plt.plot(df.date_val, trend, label=str(volume), c=colors[volume])
    plt.legend()
    
    if holiday_dates:
        for ind in df.date_val[df.is_holiday == 1]:
            plt.axvline(ind, lw=1, ls='--', c='k')
    
    
    #plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

normalize_w = widgets.Checkbox(value=True, description="Normalize")
reg_holidays_w = widgets.Checkbox(value=True, description="Regularize Holidays")
holiday_dates_w = widgets.Checkbox(value=True, description="Show Holidays")


volumes_w = widgets.SelectMultiple(options=['Volume A', 'Volume B', 'Volume C'], value=['Volume A'],)
    
plot_trends_w = widgets.interactive_output(plot_trends, dict(normalize=normalize_w,
                                                            volumes=volumes_w,
                                                            reg_holidays=reg_holidays_w,
                                                            holiday_dates=holiday_dates_w))


explore_trends_subtab = VBox([HBox([VBox([normalize_w,
                                          reg_holidays_w,
                                          holiday_dates_w]), Label(value='Select Volumes: '), volumes_w]),
                              plot_trends_w,])

## Seasonality

In [10]:
colors = {'Volume A': 'C1',
          'Volume B': 'C2',
          'Volume C': 'C3'}

def plot_seasonality(volumes, normalize=True, reg_holidays=True, holiday_dates=True):
    if len(volumes) == 0:
        return
    
    # fetch data from dataframe
    vol_a = df.volume_A.values.copy()
    vol_b = df.volume_B.values.copy()
    vol_c = df.volume_C.values.copy()

    # preprocess if requested
    if reg_holidays:
        for i in df.index[df.is_holiday == 1]:
            if i >= 7:
                source_i = i - 7
            else:
                source_i = i + 7
            vol_a[i] = vol_a[source_i]
            vol_b[i] = vol_b[source_i]
            vol_c[i] = vol_c[source_i]

    # extract trends using STL
    season_a = STL(vol_a, 7).fit().seasonal
    season_b = STL(vol_b, 7).fit().seasonal
    season_c = STL(vol_c, 7).fit().seasonal
    seasons = {'Volume A': season_a,
               'Volume B': season_b,
               'Volume C': season_c}
    
    
    plt.figure(figsize=(20, 5))
    for volume in volumes:
        season = seasons[volume]
        if normalize:
            season = normalized(season)
        plt.plot(df.date_val, season, label=str(volume), c=colors[volume])
    plt.legend()
    
    if holiday_dates:
        for ind in df.date_val[df.is_holiday == 1]:
            plt.axvline(ind, lw=1, ls='--', c='k')
    
    
    #plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

normalize_w = Checkbox(value=True, description="Normalize")
reg_holidays_w = Checkbox(value=True, description="Regularize Holidays")
holiday_dates_w = Checkbox(value=True, description="Show Holidays")


volumes_w = SelectMultiple(options=['Volume A', 'Volume B', 'Volume C'], value=['Volume A'],)
    
plot_seasonality_w = widgets.interactive_output(plot_seasonality, dict(normalize=normalize_w,
                                                                  volumes=volumes_w,
                                                                  reg_holidays=reg_holidays_w,
                                                                  holiday_dates=holiday_dates_w))


explore_seasonality_subtab = VBox([HBox([VBox([normalize_w,
                                          reg_holidays_w,
                                          holiday_dates_w]), Label(value="Select Volumes: "), volumes_w]),
                              plot_seasonality_w,])

In [11]:
colors = {'Volume A': 'C1',
          'Volume B': 'C2',
          'Volume C': 'C3'}

from statsmodels.tsa.stattools import pacf

def plot_pacf(volumes):

    # ACF Plot
    fig,ax = plt.subplots(3,1,figsize = (16,10),sharey=True)

    plt.ylim(-1,1)


    ax[0].scatter(np.arange(0,np.size(pacf(df['volume_A']))), 
               pacf(df['volume_A']) ,
               c='b',label='volume_A')
    ax[1].scatter(np.arange(0,np.size(pacf(df['volume_B']))), 
               pacf(df['volume_B']) ,
               c='r',label='volume_B')
    ax[2].scatter(np.arange(0,np.size(pacf(df['volume_C']))), 
               pacf(df['volume_C']) ,
               c='g',label='volume_C')

  
    ax[0].plot(pacf(df['volume_A']),'b')
    ax[1].plot(pacf(df['volume_B']),'r')
    ax[2].plot(pacf(df['volume_C']),'g')

    ax[0].legend(fontsize=14)
    ax[1].legend(fontsize=14)
    ax[2].legend(fontsize=14)

    plt.show()
    
plot_pacf_w = widgets.interactive_output(plot_pacf, dict(volumes=volumes_w))


explore_pacf_subtab = VBox([HBox([VBox()]),
                              plot_pacf_w,])

In [12]:
colors = {'Volume A': 'C1',
          'Volume B': 'C2',
          'Volume C': 'C3'}

def make_lag_df(df,feature,lag):
    lag_df = df.copy()
    lag_df[feature + '_lag'] = np.nan
    
    lag_df.loc[lag:,feature + '_lag'] = lag_df.loc[0:len(lag_df)-(lag+1),feature].values
    return lag_df

def get_autocorr(df,feature,lag):
    df = make_lag_df(df,feature,lag)
    mean_y = df[feature].mean()
    
    y_ts = df[feature].values
    y_lags = df.dropna()[feature + '_lag'].values
    
    numerator = np.sum((y_ts[lag:] - mean_y)*(y_lags - mean_y))
    denom = np.sum(np.power(y_ts - mean_y,2))
    
    return numerator/denom


def plot_acf(volumes):

    max_lag = 63

    # ACF Plot
    fig,ax = plt.subplots(3,1,figsize = (16,10),sharey=True)

    plt.ylim(-1,1)

    ax[0].axhline(y=0, xmin=0, xmax=20)
    ax[1].axhline(y=0, xmin=0, xmax=20)
    ax[2].axhline(y=0, xmin=0, xmax=20)

    ax[0].scatter(np.arange(1,max_lag+1,1), 
               [get_autocorr(df,'volume_A',lag) for lag in np.arange(1,max_lag+1,1)],
               c='b',label='volume_A')
    ax[1].scatter(np.arange(1,max_lag+1,1), 
               [get_autocorr(df,'volume_B',lag) for lag in np.arange(1,max_lag+1,1)],
               c='r',label='volume_B')
    ax[2].scatter(np.arange(1,max_lag+1,1), 
               [get_autocorr(df,'volume_C',lag) for lag in np.arange(1,max_lag+1,1)],
               c='g',label='volume_C')

    for i in np.arange(1,max_lag + 1,1):
        ax[0].plot(i*np.ones(2),[0,get_autocorr(df,'volume_A',i)],'b')
        ax[1].plot(i*np.ones(2),[0,get_autocorr(df,'volume_B',i)],'r')
        ax[2].plot(i*np.ones(2),[0,get_autocorr(df,'volume_C',i)],'g')

    ax[0].legend(fontsize=14)
    ax[1].legend(fontsize=14)
    ax[2].legend(fontsize=14)

    plt.show()
    
plot_acf_w = widgets.interactive_output(plot_acf, dict(volumes=volumes_w))


explore_acf_subtab = VBox([HBox([VBox()]),
                              plot_acf_w,])

## Daily Trends

In [13]:
df_cmm = df
days = ['Sun','Mon','Tue','Wed','Thu','Fri','Sat']
day_of_week = []
for i in range(len(df)):
    day_of_week.append(days[i%7])
    
df_cmm['Day_of_week'] = day_of_week

In [14]:
colors = {'Volume A': 'C1',
          'Volume B': 'C2',
          'Volume C': 'C3'}

months_arr = [0,31,59,90,120,151,181,212,243,273,304,334,365]

months_list = ['January','February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November',
          'December']

years = ['2017','2018','2019']

def plot_daily_trend(month, year):
    if len(month) == 0:
        return
    
    ind_mon = months_list.index(month)
    ind_year = years.index(year)
    
    beginning = 365*ind_year + months_arr[ind_mon]
    end = 365*ind_year + months_arr[ind_mon+1]
    indices = df_cmm.iloc[beginning:end]
    
    plt.figure(figsize=(17, 5))
    plt.plot(range(1,end - beginning+1),indices['volume_A'], label = 'volume_A')
    plt.plot(range(1,end - beginning+1),indices['volume_B'], label = 'volume_B')
    plt.plot(range(1,end - beginning+1),indices['volume_C'], label = 'volume_C')
    title = months_list[ind_mon]
    #plt.title(title + year)
    plt.grid(b=True)
    plt.xticks(range(1,end-beginning+1),labels = df_cmm['Day_of_week'].iloc[beginning:end],rotation = 90)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
          ncol=3, fancybox=True, shadow=True)
    #plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()


#volumes_w = SelectMultiple(options=['Volume A', 'Volume B', 'Volume C'], value=['Volume A'],)
months_w = Dropdown(options=['January', 'February', 'March', 'April','May','June','July','August','September','October',\
                                  'November','December'], value = 'January',)
years_w = Dropdown(options = ['2017','2018','2019'], value = '2017',)
    
plot_daily_trend_w = widgets.interactive_output(plot_daily_trend, dict(month=months_w,
                                                                  year = years_w))

explore_daily_subtab = VBox([HBox([Label(value = "Month:" ), months_w, Label(value="Year: "), years_w]),
                              plot_daily_trend_w,])

## Monthly Trends

In [23]:
colors = {'2017': 'C1',
          '2018': 'C2',
          '2019': 'C3'}

years = ['2017','2018','2019']

volumes_col = ['volume_A', 'volume_B', 'volume_C']

volumes = ['Volume A', 'Volume B', 'Volume C']

months_list = ['January','February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November',
          'December']

months_arr = [0,31,59,90,120,151,181,212,243,273,304,334,365]

def plot_monthly_trend(volume,year):
    if len(year) == 0:
        return
    ind_vol = volumes.index(volume)
    volume_target = volumes_col[ind_vol]
    
    plt.figure(figsize=(17, 6))
    for ele in year:
        ind_year = years.index(ele)
        sum_A = []
        for j in range(12):
            beginning = months_arr[j]
            end = months_arr[j+1]
            a = df_cmm.iloc[beginning+365*ind_year:end+365*ind_year][volume_target].sum()
            sum_A.append(a)
        plt.plot(range(1,13),sum_A, label=str(ele), c=colors[ele])
    plt.xticks(range(1,13), labels = months_list, rotation = 90)
    plt.xlabel('Months')
    plt.ylabel('Volume')
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
          ncol=3, fancybox=True, shadow=True)
    plt.grid(b=True)

volumes_w = Dropdown(options=['Volume A', 'Volume B', 'Volume C'], value='Volume A',)
years_w = SelectMultiple(options = ['2017','2018','2019'], value = ['2017'],)
    
plot_monthly_trend_w = widgets.interactive_output(plot_monthly_trend, dict(volume=volumes_w,
                                                                  year = years_w))


explore_monthly_subtab = VBox([HBox([Label(value = "Select Volumes:" ), volumes_w, Label(value="Year: "), years_w]),
                              plot_monthly_trend_w,])

In [24]:
explore_tab = Tab(children=[explore_trends_subtab, explore_seasonality_subtab,
                            explore_acf_subtab, explore_pacf_subtab, explore_daily_subtab,
                           explore_monthly_subtab])
explore_tab.set_title(0, "Trend")
explore_tab.set_title(1, "Seasonal ")
explore_tab.set_title(2, "ACF")
explore_tab.set_title(3, "PACF")
explore_tab.set_title(4, "Daily Trend")
explore_tab.set_title(5, "Monthly Trend")
explore_tab.set_title(6, "Yearly Trend")

# Models and Forecasting

## Seasonal Naive Method

In [25]:
# naive sesonal method for benchmarking
def naive_forecast(volume, horizon='1 year'):
    period = 7
    season_to_repeat = 3
    
    train_sets_, test_sets_, _ = get_train_test(test_start_dates[horizon])
    
    steps = len(test_sets_[volume])
    train = train_sets_[volume]
    
    index_base = len(train) - season_to_repeat*period
    indices = index_base + np.arange(0, steps, dtype=int)%period
    return train[indices]

In [26]:
horizon_shorter_name = {'1 month': '1m',
                        '3 months': '3m',
                        '6 months': '6m',
                        '1 year': '12m'}

def get_sarimax_filename(volume, horizon):
    return "vasudha/" + volume[-1] + "_" + horizon_shorter_name[horizon] + "_sarimax.csv"

def get_arima_filename(volume, horizon):
    return "kritika/" + volume[-1] + "_" + horizon_shorter_name[horizon] + "_arima.csv"

def get_svarmax_filename(volume, horizon):
    return "svarmax/" + volume[-1] + "_" + horizon_shorter_name[horizon] + "_svarmax.csv"


volumes = ['Volume A', 'Volume B', 'Volume C']
horizons = ['1 month', '3 months', '6 months', '1 year']

sarimax_forecasts = {}
arima_forecasts = {}
svarmax_forecasts = {}

# confidence intervals
sarimax_lower = {}
sarimax_upper = {}

arima_lower = {}
arima_upper = {}

svarmax_lower = {}
svarmax_upper = {}

for volume in volumes:
    sarimax_forecasts[volume] = {}
    sarimax_upper[volume] = {}
    sarimax_lower[volume] = {}
    
    arima_forecasts[volume] = {}
    arima_upper[volume] = {}
    arima_lower[volume] = {}
    
    svarmax_forecasts[volume] = {}
    svarmax_upper[volume] = {}
    svarmax_lower[volume] = {}
    
    for horizon in horizons:
        csv_df = pd.read_csv(get_sarimax_filename(volume, horizon),
                             header=None, names=["forecast", "lower", "upper"])

        sarimax_forecasts[volume][horizon] = csv_df.forecast
        sarimax_upper[volume][horizon] = csv_df.upper
        sarimax_lower[volume][horizon] = csv_df.lower
        

        csv_df = pd.read_csv(get_arima_filename(volume, horizon),
                             header=None, names=["forecast", "lower", "upper"])

        arima_forecasts[volume][horizon] = csv_df.forecast
        arima_upper[volume][horizon] = csv_df.upper
        arima_lower[volume][horizon] = csv_df.lower
        
        csv_df = pd.read_csv(get_svarmax_filename(volume, horizon),
                             header=None, names=["forecast", "lower", "upper"])

        svarmax_forecasts[volume][horizon] = csv_df.forecast
        svarmax_upper[volume][horizon] = csv_df.upper
        svarmax_lower[volume][horizon] = csv_df.lower

In [27]:
colors = {'Volume A': 'C1',
          'Volume B': 'C2',
          'Volume C': 'C3'}

models_dic = {'Seasonal Naive Method': 'snm',
              'SARIMAX': 'sarimax',
              'sVARMAX': 'svarmax',
              'ARIMA': 'arima'}




def get_forecast(volume, horizon, model):
    model_codename = models_dic[model]
    
    if model_codename is "snm":
        return naive_forecast(volume, horizon)
    elif model_codename is "sarimax":
        return sarimax_forecasts[volume][horizon]
    elif model_codename is "arima":
        return arima_forecasts[volume][horizon]
    elif model_codename is "svarmax":
        return svarmax_forecasts[volume][horizon]
    else:
        return np.zeros_like(naive_forecast(volume, horizon))

def get_confid_int(volume, horizon, model):
    model_codename = models_dic[model]
    
    if model_codename is "sarimax":
        return sarimax_lower[volume][horizon], sarimax_upper[volume][horizon]
    elif model_codename is "arima":
        return arima_lower[volume][horizon], arima_upper[volume][horizon]
    else:
        return None, None


def plot_forecast(volumes, horizon, models, merge=False):
    if len(volumes) == 0:
        return
    
    train_sets, test_sets, test_date_vals = get_train_test(test_start_dates[horizon])
    ax_height = 3
    if merge:
        num_rows = 2
        fig, axs = plt.subplots(num_rows, 1, figsize=(15, num_rows * ax_height), sharex=True)
        forecast_ax = axs[0]
        data_ax = axs[0]
        err_ax = axs[1]
    else:
        num_rows = 3
        fig, axs = plt.subplots(num_rows, 1, figsize=(15, num_rows * ax_height), sharex=True)
        forecast_ax = axs[0]
        data_ax = axs[1]
        err_ax = axs[2]

    
    for volume in volumes:
        train = train_sets[volume]
        test = test_sets[volume]
        
        forecasts = np.array([get_forecast(volume, horizon, model) for model in models])
        forecast = np.mean(forecasts, axis=0)
        ape = 100.0 * np.abs(forecast - test) / test

        forecast_ax.plot(test_date_vals, forecast, color=colors[volume], ls='--', lw=2, label=volume)
        data_ax.plot(test_date_vals, test, color=colors[volume], lw=1, label=volume)
        
        if len(models) == 1:
            for model in models:
                confid_lower, confid_upper = get_confid_int(volume, horizon, model)
                if confid_upper is not None:
                    forecast_ax.fill_between(test_date_vals, confid_lower, confid_upper, alpha=.3)
        
        err_ax.plot(test_date_vals, ape, color=colors[volume], label=volume)
        err_ax.set_ylabel("Absolute Percent Error")
        
    plt.legend()
    plt.xticks(rotation=20)
    plt.tight_layout()
    plt.show()
    
    print(fr"Mean absolute percent error: {np.mean(ape):.2f}")

    
 
horizon_label_w = Label(value="Forecast horizon: ")
horizon_w = Dropdown(options=['1 month', '3 months', '6 months', '1 year'], value='1 year',)

volumes_label_w = Label(value="Volumes: ")
volumes_w = SelectMultiple(options=['Volume A', 'Volume B', 'Volume C'], value=['Volume A'],)

models_label_w = Label(value="Model(s): ")
models_w = SelectMultiple(options=models_dic.keys(),
                                 value=['Seasonal Naive Method'],)

merge_plots_w = Checkbox(value=False, description="Merge forecast and test plots")


plot_forecast_w = widgets.interactive_output(plot_forecast, dict(volumes=volumes_w,
                                                                 horizon=horizon_w,
                                                                 models=models_w,
                                                                 merge=merge_plots_w))

forecast_subtab = VBox([HBox([volumes_label_w, volumes_w,
                              VBox([HBox([horizon_label_w, horizon_w]), merge_plots_w]),
                                    models_label_w, models_w]),
                              plot_forecast_w,])

# SARIMA

In [28]:
models_tab = Tab(children=[forecast_subtab])
models_tab.set_title(0, "Forecast and Error")

# Summary

In [29]:
tabs = Tab(children=[explore_tab, models_tab])
tabs.set_title(0, "Explore Data")
tabs.set_title(1, "Models & Forecasts")
display(tabs)

effect of B on A:
lag: 1 2 ... 
p:   4e-4 0 ...
effect of C on A:
3e-3 0 ...
effect of B on c
